# Q3: Even deeper! Resnet18 for PASCAL classification (15 pts)

Hopefully we all got much better accuracy with the deeper model! Since 2012, much deeper architectures have been proposed. [ResNet](https://arxiv.org/abs/1512.03385) is one of the popular ones. In this task, we attempt to further improve the performance with the “very deep” ResNet-18 architecture.


## 3.1 Build ResNet-18 (1 pts)
Write a network modules for the Resnet-18 architecture (refer to the original paper). You can use `torchvision.models` for this section, so it should be very easy! 
Do not load the pretrained weights for this question. We will get to that in the next question.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import matplotlib.pyplot as plt
%matplotlib inline

import trainer
from utils import ARGS
from simple_cnn import SimpleCNN
from voc_dataset import VOCDataset


# you could write the whole class....
# or one line :D
ResNet = models.resnet18(pretrained=False)
# # print(model)
# ResNet = torch.nn.Sequential(*(list(model.children())[:-1]))
# # print(ResNet)
# ResNet.add_module("fc", nn.Linear(512, 20))
# print(ResNet)
in_features = ResNet.fc.in_features
ResNet.fc = nn.Sequential(nn.Linear(in_features, 20))

## 3.2 Add Tensorboard Summaries (6 pts)
You should've already written tensorboard summary generation code into `trainer.py` from q1. However, you probably just added the most basic summary features. Please implement the more advanced summaries listed here:
* training loss (should be done)
* testing MAP curves (should be done)
* learning rate
* [histogram of gradients](https://www.tensorflow.org/api_docs/python/tf/summary/histogram)

## 3.3 Train and Test (8 pts)
Use the same hyperparameter settings from Task 2, and train the model for 50 epochs. Tune hyperparameters properly to get mAP around 0.5. Report tensorboard screenshots for *all* of the summaries listed above (for image summaries show screenshots at $n \geq 3$ iterations). For the histograms, include the screenshots of the gradients of layer1.1.conv1.weight and layer4.0.bn2.bias.

**REMEMBER TO SAVE A MODEL AT THE END OF TRAINING**

In [3]:
args = ARGS(epochs=50, lr=0.0001, batch_size=32, save_at_end=True, save_freq=10, step_size=30, test_batch_size=128, use_cuda=True, gamma=0.1, val_every=250)
resnet = ResNet
print(args)
optimizer = torch.optim.Adam(resnet.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
test_ap, test_map = trainer.train(args, resnet, optimizer, scheduler, model_name='resnet2')
print('test map:', test_map)

args.batch_size = 32
args.device = cuda
args.epochs = 50
args.gamma = 0.1
args.inp_size = 224
args.log_every = 100
args.lr = 0.0001
args.save_at_end = True
args.save_freq = 10
args.step_size = 30
args.test_batch_size = 128
args.val_every = 250

Train Epoch: 0 [0 (0%)]	Loss: 0.699209
Train Epoch: 0 [100 (64%)]	Loss: 0.202564
Train Epoch: 1 [200 (27%)]	Loss: 0.222251
Train Epoch: 1 [300 (91%)]	Loss: 0.205923
Train Epoch: 2 [400 (55%)]	Loss: 0.214943
Train Epoch: 3 [500 (18%)]	Loss: 0.189991
Train Epoch: 3 [600 (82%)]	Loss: 0.168802
Train Epoch: 4 [700 (46%)]	Loss: 0.192032
Train Epoch: 5 [800 (10%)]	Loss: 0.191507
Train Epoch: 5 [900 (73%)]	Loss: 0.183262
Train Epoch: 6 [1000 (37%)]	Loss: 0.178853
Train Epoch: 7 [1100 (1%)]	Loss: 0.170648
Train Epoch: 7 [1200 (64%)]	Loss: 0.186747
Train Epoch: 8 [1300 (28%)]	Loss: 0.173882
Train Epoch: 8 [1400 (92%)]	Loss: 0.167580
Train Epoch: 9 [1500 (55%)]	Loss: 0.146841
Train Epoch: 10 [1600 (19%)]	Loss: 0.141675
Train Epoch: 10 [1700 (83%)]	Loss: 0.

***Loss for training***

<img src="vlr-hw1-images/q3-loss.png"/>


***mAP for testing*** 

<img src="vlr-hw1-images/q3-map.png"/>

***Learning Rate*** 

<img src="vlr-hw1-images/q3-lr.png"/>


***Histogram layer1.1.conv1.weight***

<img src="vlr-hw1-images/q3-hist-conv1.png"/>


***Histogram layer4.0.bn2.bias*** 

<img src="vlr-hw1-images/q3-hist-bias.png"/>